In [80]:
from connect_db import connect_to_grad_db
import pandas as pd
import re
import math

In [2]:
engine = connect_to_grad_db()

In [5]:
reader = pd.read_sql("news_api_w_preds", engine.connect(), chunksize=1)

In [6]:
df = reader.__next__()

In [10]:
text_w_preds = df["text_w_preds"].iloc[0]

In [24]:
sentences = re.split(r"\. ", text_w_preds)

In [192]:
banners = {
    "fox-news":"https://s1.qwant.com/thumbr/0x380/9/6/9484111a9e2eb4cbb8ede5d0bc606c378ed898aeb7f1bd2dd64a13b79b7d16/1200px-Fox_News_Channel_logo.svg.png?u=https%3A%2F%2Fupload.wikimedia.org%2Fwikipedia%2Fcommons%2Fthumb%2F6%2F67%2FFox_News_Channel_logo.svg%2F1200px-Fox_News_Channel_logo.svg.png&q=0&b=1&p=0&a=0",
    "abc-news":"https://s1.qwant.com/thumbr/700x0/3/e/0247f90b2a3859f30c2f2aa3bf8c0a4a3bb5fb50a4356a0fabc82cb6a014b6/latest.jpg?u=https%3A%2F%2Fvignette4.wikia.nocookie.net%2Flogopedia%2Fimages%2Fe%2Fe1%2FABC_News.jpg%2Frevision%2Flatest%3Fcb%3D20151013161111&q=0&b=1&p=0&a=0",
    "cnn":"https://s2.qwant.com/thumbr/700x0/f/e/fb9962b40abff59c434b2e63eb63d72021cb7912e6f784bd2388fd21ff67c4/cnn-logo-1-1.png?u=https%3A%2F%2Flogodownload.org%2Fwp-content%2Fuploads%2F2014%2F11%2Fcnn-logo-1-1.png&q=0&b=1&p=0&a=0",
    "al-jazeera-english":"https://s2.qwant.com/thumbr/0x380/6/e/8509f372409f486f2279183019365bd9b0599d3b4c01dd6829861561e0e77e/JAZEERA-LOGO.jpg?u=http%3A%2F%2Fwww.wired.com%2Fwp-content%2Fuploads%2Fimages_blogs%2Fbusiness%2F2011%2F01%2FJAZEERA-LOGO.jpg&q=0&b=1&p=0&a=0"

}

In [134]:
def decimal_to_rbg(decimal):
    r = int((4-decimal)*255/4)
    g = int((decimal)*255/4)
    b = 120


    return f"rgb({r}, {g}, {b})"

In [170]:
html_sentences = []
for sentence in sentences:
    pred = float(re.search(r"\[\[(.*)\]\]", sentence).group(1))

    pred_color = decimal_to_rbg(pred)
    html_text = fr'<t style="background-color:{pred_color};">{sentence}.</t>'

    html_sentences.append(html_text)

In [200]:
title = df["title"].iloc[0]
title = f"<h1>{title}</h1>"

date= df["publishedAt"].iloc[0]
date = f"<p>Published: {date}</p>"

author = df["author"].iloc[0]
author = f"<p>Written by: {author}</p>"

mean_pred = df["mean_preds"].iloc[0]
mean_pred = f"<b style='background-color:{decimal_to_rbg(mean_pred)};padding:0px;'>{mean_pred}</b>"

source = df["source"].iloc[0]
banner = f"<img src='{banners[source]}' style='height:100px;'>"


In [201]:
with open("html_test.html", "w") as file:
    file.write("<!DOCTYPE html>")
    file.write("<html>")
    file.write(banner)
    file.write(title)
    file.write(f"<p>Source: {source}</p>")
    file.write(date)
    file.write(author)
    file.write(f"Sentiment overall: <br>{mean_pred}<br><br>")
    file.write(
"""
<style>

    t {
        font-size: large;
    }

    html {
        margin:0 450px;
        text-align: center;
        font-family: system-ui, -apple-system, BlinkMacSystemFont, 'Segoe UI', Roboto, Oxygen, Ubuntu, Cantarell, 'Open Sans', 'Helvetica Neue', sans-serif;
    }



</style>
""")
    file.write("".join(html_sentences))
    file.write("</html>")